1. Crear un DataFrame que tenga la variación porcentual anual de personas para las categorías
independientes (total) y asalariados (total).

In [242]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as make_subplots

#importamos las bases
DF_EMP = pd.read_excel('EMPLEO.xlsx')
DF_EXP = pd.read_excel('EXP.xlsx')
DF_IMP = pd.read_excel('IMP.xlsx')
DF_IMACEC = pd.read_excel('IMACEC.xlsx')

#filtramos los datos
filterTrimestre = DF_EMP['Trimestre'] == 'Dic - Feb'
df = DF_EMP[filterTrimestre]

#preparamos los datos
ano = df['Ano']
independientes = df['Independientes']
asalariados = df['Asalariados']
variacion = {
    'Ano' : ano,
    'independientes' : independientes,
    'asalariados' : asalariados
}

#creamos el DF a partir del dict
variacionDF = pd.DataFrame.from_dict(variacion)

#Calculamos variacion anual 
variacionDF['independientes'] = independientes.pct_change()
variacionDF['asalariados'] = asalariados.pct_change()

2. Usando el DataFrame del IMACEC, transponer para que las fechas queden en las filas y las
categorías como columnas. Para esto puede usar la función de pandas pivot_table .

In [243]:
transpose = DF_IMACEC.pivot_table(columns=['Descrip'])

3. Al DF del punto 1), agregar la variación porcentual anual del imacec del punto 2), usando un 
inner join .


In [246]:
#preparamos los datos
measures = {
    'Ano' : [],
    'Imacec' : []
}

dates = transpose.index
for date in dates:
    if(date.day == 1 and date.month == 1):
        measures['Imacec'].append(transpose.loc[date]['Imacec'])
        measures['Ano'].append(date.year)
        pass

#creammos el DF a partir del dict 
imacecDF = pd.DataFrame.from_dict(measures)

#calculamos la variacion
imacecDF['Imacec'] = imacecDF['Imacec'].pct_change()

#creamos el join
innerJoin = pd.merge(imacecDF,variacionDF,on='Ano')


#El primer valor de Imacec no es NaN, a diferencia de indep. y asalariados
innerJoin.loc[innerJoin.Ano == '2011', 'Imacec']  = float('nan')

4. Utilizando plotly , crear un subplot que contenga: 

1) Gráfico con la variación anual deindependientes, asalariados e IMACEC. Debe usar líneas, markes o dash para diferenciar las tres líneas. 

2) Usando los DataFrame de exportaciones (EXP) e importaciones (IMP), crear un gráfico de scatter que tenga el total de exportaciones en el eje horizontal y el total de importaciones (CIF) en el vertical. Agregue al gráfico una línea de 45°. 

3) Dentro de las exportaciones seleccione: total exportaciones, minería,  agropecuario-silvícola y pesquero, industriales. 
Utilizando la primera fecha (enero-2009) como base 100, crear un mapa de calor que en el eje horizontal tenga las fechas y en el vertical las 4 categorías. Cambie la escala de colores (no utilizar el por defecto).

In [263]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

'''
graph1
'''
fig = make_subplots(rows=2, cols=2)

#preparamos los datos
imacecData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["Imacec"],
    name="Imacec")

asalariadosData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["asalariados"],
    name="Asalariados")

indepData = go.Scatter(
    x=innerJoin['Ano'],
    y=innerJoin["independientes"],
    name="Independientes")

data = [imacecData, asalariadosData, indepData]

#graficamos
layout = go.Layout(
    yaxis=dict(
        domain=[0, 1]
    ),
    legend=dict(
        traceorder="reversed"
    ),
    yaxis2=dict(
        domain=[0.33, 0.66]
    ),
    yaxis3=dict(
        domain=[0.66, 1]
    ),
    width=2000, height=800,
    title = "Variacion Anual de independientes, asalariados e Imacec",
    xaxis_title="Fecha",
    yaxis_title="Variacion Anual",
)

fig = go.Figure(data=data, layout=layout)
fig.show()

'''
graph2
'''
#filtramos los datos
impData = DF_IMP.iloc[0].iloc[2:]
expData = DF_EXP.iloc[0].iloc[2:]

#preparamos los datos
dataDict = {
    'imp' : impData,
    'exp' : expData
}
dataDF = pd.DataFrame.from_dict(dataDict)

#graficamos
fig2 = px.scatter(
    dataDF,
    x="exp",
    y="imp",
    title = "Scatter Exportaciones vs Importaciones ",
    width=800, height=800
)
fig2.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1}])
fig2.show()

'''
graph3
'''
#filtramos los datos
dataRaw = DF_EXP.loc[[0,1,11,26]]

#preparamos los datos
heatMapDict = {
    'categorias' : dataRaw['Descripción series'],
    'fechas' : dataRaw.columns[2:]
}

#graficamos
fig3 = px.imshow(
    dataRaw,
    x=dataRaw.columns,
    y=dataRaw['Descripción series'],
    text_auto=True,
    aspect="auto",
    labels=dict(x="Fechas", y="Categorias", color="Exportaciones"),
    width=1600, height=800,
    color_continuous_scale='turbo'
    
    )
fig3.show()

